In [24]:
!pip install psycopg2

In [47]:
import pandas as pd
from configparser import ConfigParser
import psycopg2
import pandas.io.sql as sqlio
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from pathlib import Path
from collections import Counter
from sklearn import preprocessing
from sklearn import utils

In [26]:
##Note: uncomment this if you are using a config.py file, and change the "password"
# from config import password
password = "Post23!Fnord"

In [27]:
param_dic={
    "host": "happinessproject.cqkpnj5dcmou.us-east-1.rds.amazonaws.com",
    "database": "project-data",
    "user" : "postgres",
    "password" : password
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [28]:
query="select * from worldhappinessreport_history"

worldhappinessreport_2021_df=sqlio.read_sql_query(query,conn)

worldhappinessreport_2021_df.head()

C:\Users\Brittany\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,country_name,year_num,life_ladder,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,positive_affect,negative_affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [29]:
query1="select * from screen_time"

screen_time_2021_df=sqlio.read_sql_query(query1,conn)

screen_time_2021_df.head()

C:\Users\Brittany\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,country_name_clean,country_name_source,screen_time_avg
0,South Africa,South Africa,10.46
1,Philippines,Philippines,10.27
2,Brazil,Brazil,10.19
3,Colombia,Colombia,10.03
4,Argentina,Argentina,9.38


In [30]:
screen_time_2021_df.columns =['country_name', 'country_name_source', 'screen_time_avg']
screen_time_2021_df.head()

,country_name,country_name_source,screen_time_avg
0,South Africa,South Africa,10.46
1,Philippines,Philippines,10.27
2,Brazil,Brazil,10.19
3,Colombia,Colombia,10.03
4,Argentina,Argentina,9.38


In [31]:
happiness2021_screen_df = worldhappinessreport_2021_df.merge(screen_time_2021_df , on='country_name', how = 'left')
happiness2021_screen_df.head()

,country_name,year_num,life_ladder,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,positive_affect,negative_affect,country_name_source,screen_time_avg
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258,NaN,NaN
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237,NaN,NaN
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275,NaN,NaN
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267,NaN,NaN
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268,NaN,NaN


In [32]:
query2="select * from land_area"

land_area_df=sqlio.read_sql_query(query2,conn)

land_area_df.head()

C:\Users\Brittany\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,country_name,population_2020,land_area_skm,density_skm
0,Afghanistan,38041750.0,652230.0,56.94
1,Albania,2854190.0,28748.0,104.61
2,Algeria,43053050.0,2381740.0,17.73
3,Andorra,77140.0,468.0,163.84
4,Angola,31825290.0,1246700.0,24.71


In [33]:
happiness2021_screen_land_df = happiness2021_screen_df.merge(land_area_df , on='country_name', how = 'left')
happiness2021_screen_land_df.head()

,country_name,year_num,life_ladder,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,positive_affect,negative_affect,country_name_source,screen_time_avg,population_2020,land_area_skm,density_skm
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258,NaN,NaN,38041750.0,652230.0,56.94
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237,NaN,NaN,38041750.0,652230.0,56.94
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275,NaN,NaN,38041750.0,652230.0,56.94
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267,NaN,NaN,38041750.0,652230.0,56.94
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268,NaN,NaN,38041750.0,652230.0,56.94


In [34]:
happiness2021_screen_land_df.columns

Index(['country_name', 'year_num', 'life_ladder', 'log_gdp_per_cap',
       'social_support', 'healty_life_exp', 'freedom', 'generosity',
       'percept_corrupt', 'positive_affect', 'negative_affect',
       'country_name_source', 'screen_time_avg', 'population_2020',
       'land_area_skm', 'density_skm'],
      dtype='object')

In [35]:
happiness2021_screen_land_df.dtypes

country_name            object
year_num                 int64
life_ladder            float64
log_gdp_per_cap        float64
social_support         float64
healty_life_exp        float64
freedom                float64
generosity             float64
percept_corrupt        float64
positive_affect        float64
negative_affect        float64
country_name_source     object
screen_time_avg        float64
population_2020        float64
land_area_skm          float64
density_skm            float64
dtype: object

In [36]:
happiness2021_screen_land_df.describe

<bound method NDFrame.describe of      country_name  year_num  life_ladder  log_gdp_per_cap  social_support  \
0     Afghanistan      2008        3.724            7.370           0.451   
1     Afghanistan      2009        4.402            7.540           0.552   
2     Afghanistan      2010        4.758            7.647           0.539   
3     Afghanistan      2011        3.832            7.620           0.521   
4     Afghanistan      2012        3.783            7.705           0.521   
...           ...       ...          ...              ...             ...   
1944     Zimbabwe      2016        3.735            7.984           0.768   
1945     Zimbabwe      2017        3.638            8.016           0.754   
1946     Zimbabwe      2018        3.616            8.049           0.775   
1947     Zimbabwe      2019        2.694            7.950           0.759   
1948     Zimbabwe      2020        3.160            7.829           0.717   

      healty_life_exp  freedom  generosit

In [37]:
happiness2021_screen_land_df.isna().sum().sum()

3434

In [38]:
happiness2021_screen_land_df.isna().sum()

country_name              0
year_num                  0
life_ladder               0
log_gdp_per_cap          36
social_support           13
healty_life_exp          55
freedom                  32
generosity               89
percept_corrupt         110
positive_affect          22
negative_affect          16
country_name_source    1337
screen_time_avg        1337
population_2020         129
land_area_skm           129
density_skm             129
dtype: int64

In [39]:
# Drop the null columns where all values are null
happiness2021_screen_land_df = happiness2021_screen_land_df.dropna(axis='columns', how='all')

# Drop the null rows
happiness2021_screen_land_df = happiness2021_screen_land_df.dropna()

In [40]:
happiness2021_screen_land_df.isna().sum()

country_name           0
year_num               0
life_ladder            0
log_gdp_per_cap        0
social_support         0
healty_life_exp        0
freedom                0
generosity             0
percept_corrupt        0
positive_affect        0
negative_affect        0
country_name_source    0
screen_time_avg        0
population_2020        0
land_area_skm          0
density_skm            0
dtype: int64

In [60]:
happiness2021_screen_land_df.head()

,country_name,year_num,life_ladder,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,positive_affect,negative_affect,country_name_source,screen_time_avg,population_2020,land_area_skm,density_skm
0,Argentina,2006,6.313,9.942,0.938,66.82,0.733,-0.157,0.852,0.825,0.328,Argentina,9.38,44938710.0,2780400.0,16.26
1,Argentina,2007,6.073,10.018,0.862,66.94,0.653,-0.141,0.881,0.828,0.279,Argentina,9.38,44938710.0,2780400.0,16.26
2,Argentina,2008,5.961,10.048,0.892,67.06,0.678,-0.132,0.865,0.823,0.318,Argentina,9.38,44938710.0,2780400.0,16.26
3,Argentina,2009,6.424,9.977,0.919,67.18,0.637,-0.130,0.885,0.864,0.237,Argentina,9.38,44938710.0,2780400.0,16.26
4,Argentina,2010,6.441,10.066,0.927,67.30,0.730,-0.126,0.855,0.846,0.211,Argentina,9.38,44938710.0,2780400.0,16.26


# Random Forest, Figure out top 10 variables

In [59]:
# Create our features
X = pd.get_dummies(happiness2021_screen_land_df.drop(columns="life_ladder"))

# Create our target
y = happiness2021_screen_land_df["life_ladder"]



In [54]:
X.describe()

,year_num,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,positive_affect,negative_affect,screen_time_avg,...,country_name_source_South Africa,country_name_source_South Korea,country_name_source_Spain,country_name_source_Sweden,country_name_source_Switzerland,country_name_source_Thailand,country_name_source_Turkey,country_name_source_UAE,country_name_source_UK,country_name_source_Vietnam
count,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,...,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000
mean,2013.344064,10.240193,0.878489,68.740282,0.801736,0.030726,0.671680,0.756913,0.248791,7.078813,...,0.028169,0.028169,0.028169,0.028169,0.020121,0.030181,0.028169,0.006036,0.028169,0.020121
std,4.108450,0.700754,0.078969,5.137140,0.124141,0.192871,0.240931,0.091068,0.067380,1.672570,...,0.165622,0.165622,0.165622,0.165622,0.140555,0.171258,0.165622,0.077536,0.165622,0.140555
min,2005.000000,8.204000,0.511000,48.640000,0.369000,-0.335000,0.035000,0.384000,0.107000,4.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2010.000000,9.832000,0.846000,65.940000,0.739000,-0.122000,0.469000,0.697000,0.200000,6.040000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2013.000000,10.488000,0.903000,71.060000,0.834000,-0.016000,0.764000,0.777000,0.236000,6.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2017.000000,10.773000,0.935000,72.480000,0.901000,0.163000,0.861000,0.824000,0.296000,8.370000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2020.000000,11.490000,0.987000,77.100000,0.970000,0.555000,0.974000,0.934000,0.483000,10.460000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [55]:
#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

In [56]:
# Check the balance of our target values
y.value_counts()

7.257    3
6.375    3
5.514    2
5.767    2
4.720    2
        ..
4.989    1
4.522    1
5.146    1
5.027    1
5.467    1
Name: life_ladder, Length: 452, dtype: int64

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({5.023: 2,
         5.959: 2,
         7.025: 1,
         7.104: 1,
         7.118: 1,
         5.984: 1,
         4.947: 1,
         6.025: 1,
         4.977: 1,
         6.962: 1,
         5.422: 1,
         5.08: 1,
         6.378: 2,
         6.199: 2,
         7.037: 1,
         7.594: 2,
         5.332: 1,
         7.28: 1,
         6.513: 1,
         6.007: 1,
         7.287: 1,
         5.709: 1,
         6.959: 1,
         7.25: 1,
         6.086: 1,
         6.547: 1,
         7.314: 1,
         7.195: 1,
         7.029: 1,
         5.845: 1,
         6.91: 1,
         6.263: 1,
         7.683: 1,
         5.914: 1,
         7.516: 1,
         5.158: 2,
         6.375: 2,
         7.56: 1,
         5.101: 1,
         6.68: 1,
         6.475: 1,
         6.138: 1,
         5.885: 1,
         6.515: 1,
         7.137: 1,
         6.449: 1,
         4.514: 1,
         7.433: 1,
         4.225: 1,
         4.989: 1,
         7.488: 1,
         6.09: 1,
         6.234: 1,


In [61]:
X_train.shape

(372, 91)

In [58]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=10, random_state=78) 

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

ValueError: Unknown label type: 'continuous'